In [2]:
import import_ipynb
from maze_common import *
import time
import random as rnd
import numpy as np
import copy

importing Jupyter notebook from maze_common.ipynb


In [9]:
maze = Maze(75, .30)
solvable = maze.isSolvable()
print("Is solvable?", solvable)
print("Obstacles?", maze.obstacles.shape[0])

Is solvable? True
Obstacles? 1730


In [10]:
if (solvable):
    startTime = time.time()
    shortestPath, expandedCells = shortestPathSearch(maze)
    print("Expanded cells:", expandedCells)
    print("Shortest path length:", len(shortestPath), "\nShortest path:", shortestPath)
    for coords in shortestPath:
        row, col = coords
        if (maze.board[row, col] == -1):
            print("Error")

Expanded cells: 3779
Shortest path length: 149 
Shortest path: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 5), (1, 6), (2, 6), (3, 6), (3, 7), (3, 8), (4, 8), (5, 8), (5, 9), (6, 9), (7, 9), (7, 10), (7, 11), (8, 11), (8, 12), (9, 12), (10, 12), (11, 12), (11, 13), (12, 13), (13, 13), (14, 13), (15, 13), (16, 13), (16, 14), (17, 14), (18, 14), (19, 14), (20, 14), (20, 15), (21, 15), (22, 15), (22, 16), (23, 16), (24, 16), (25, 16), (26, 16), (26, 17), (27, 17), (28, 17), (29, 17), (29, 18), (29, 19), (29, 20), (29, 21), (30, 21), (30, 22), (30, 23), (30, 24), (31, 24), (31, 25), (31, 26), (31, 27), (31, 28), (32, 28), (32, 29), (33, 29), (34, 29), (35, 29), (35, 30), (35, 31), (36, 31), (36, 32), (37, 32), (38, 32), (38, 33), (38, 34), (39, 34), (39, 35), (39, 36), (39, 37), (39, 38), (40, 38), (41, 38), (42, 38), (42, 39), (43, 39), (44, 39), (45, 39), (46, 39), (47, 39), (48, 39), (49, 39), (50, 39), (51, 39), (52, 39), (52, 40), (53, 40), (54, 40), (55, 40), (55, 41), (55, 

In [11]:
def manhattanDistance(coords, maze, visited):
    cellRow, cellCol = coords
    return ((maze.dim - 1) - cellRow) + ((maze.dim - 1) - cellCol), 0

if (solvable):
    startTime = time.time()
    shortestPath, expandedCells = shortestPathSearch(maze, heuristicFunction = manhattanDistance)
    print("Expanded cells:", expandedCells)
    print("Shortest path length:", len(shortestPath), "\nShortest path:", shortestPath)
    for coords in shortestPath:
        row, col = coords
        if (maze.board[row, col] == -1):
            print("Error")

Expanded cells: 583
Shortest path length: 149 
Shortest path: [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (9, 1), (10, 1), (10, 2), (11, 2), (11, 3), (11, 4), (11, 5), (12, 5), (12, 6), (12, 7), (12, 8), (12, 9), (13, 9), (14, 9), (14, 10), (15, 10), (15, 11), (16, 11), (16, 12), (17, 12), (18, 12), (19, 12), (19, 13), (19, 14), (20, 14), (21, 14), (21, 15), (22, 15), (22, 16), (23, 16), (24, 16), (25, 16), (25, 17), (26, 17), (27, 17), (28, 17), (29, 17), (29, 18), (29, 19), (29, 20), (29, 21), (30, 21), (30, 22), (30, 23), (31, 23), (31, 24), (31, 25), (31, 26), (31, 27), (32, 27), (32, 28), (32, 29), (33, 29), (34, 29), (35, 29), (35, 30), (35, 31), (36, 31), (36, 32), (37, 32), (38, 32), (38, 33), (38, 34), (39, 34), (40, 34), (41, 34), (41, 35), (41, 36), (42, 36), (42, 37), (42, 38), (42, 39), (43, 39), (44, 39), (45, 39), (46, 39), (47, 39), (48, 39), (49, 39), (50, 39), (51, 39), (52, 39), (53, 39), (53, 40), (54, 40), (55, 40), (55, 41), (5

In [12]:
def thinMaze(maze, fractionRemove):
    thinMaze = Maze(1, 0)
    thinMaze.board = copy.deepcopy(maze.board)
    thinMaze.obstacles = copy.deepcopy(maze.obstacles)
    thinMaze.dim = maze.dim
    numRemove = round(fractionRemove * thinMaze.obstacles.shape[0])
    for i in range(0,numRemove):
        indexRemove = int(rnd.random() * thinMaze.obstacles.shape[0])
        obstacleX, obstacleY = thinMaze.obstacles[indexRemove]
        thinMaze.obstacles = np.delete(thinMaze.obstacles,indexRemove, axis=0)
        thinMaze.board[obstacleX, obstacleY] = Cell.OPEN
    return thinMaze
    
thinnedMaze = thinMaze(maze, .5)
print("Original obstacles: ", maze.obstacles.shape[0])
print("Thinned obstacles: ", thinnedMaze.obstacles.shape[0])
# print("Thinned maze:\n", thinnedMaze.board)

Original obstacles:  1730
Thinned obstacles:  865


In [13]:
def thinnedMazeShortestPathLength(coords, maze, visited):
    cellRow, cellCol = coords
    shortestPathLength = 0
    expandedCells = 0
    if ((cellRow, cellCol)) not in visited:
        shortestPath, expandedCellsBySearch = shortestPathSearch(thinnedMaze, startCoords = (cellRow, cellCol))
        shortestPathLength = len(shortestPath)
        expandedCells = expandedCellsBySearch
        for i in range(0, shortestPathLength):
            row, col = shortestPath[i]
            if ((row, col)) in visited: 
                break
            else:
                visited[(row, col)] = shortestPathLength - i - 1
    else:
        shortestPathLength = visited[(cellRow, cellCol)]
    return shortestPathLength, expandedCells

if (solvable):
    startTime = time.time()
    shortestPath, expandedCells = shortestPathSearch(maze, heuristicFunction = thinnedMazeShortestPathLength)
    print("Time:", time.time() - startTime, "seconds")
    print("Expanded cells:", expandedCells)
    print("Shortest path length:", len(shortestPath), "\nShortest path:", shortestPath)
    for coords in shortestPath:
        row, col = coords
        if (maze.board[row, col] == -1):
            print("Error")

Time: 15.009394645690918 seconds
Expanded cells: 694301
Shortest path length: 149 
Shortest path: [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (10, 1), (10, 2), (11, 2), (11, 3), (12, 3), (12, 4), (12, 5), (12, 6), (12, 7), (12, 8), (12, 9), (13, 9), (13, 10), (13, 11), (14, 11), (15, 11), (16, 11), (16, 12), (17, 12), (18, 12), (19, 12), (19, 13), (19, 14), (20, 14), (21, 14), (22, 14), (22, 15), (22, 16), (23, 16), (24, 16), (25, 16), (26, 16), (26, 17), (27, 17), (27, 18), (28, 18), (29, 18), (29, 19), (29, 20), (29, 21), (30, 21), (30, 22), (30, 23), (31, 23), (31, 24), (31, 25), (31, 26), (31, 27), (32, 27), (32, 28), (32, 29), (33, 29), (34, 29), (35, 29), (35, 30), (35, 31), (36, 31), (36, 32), (37, 32), (38, 32), (38, 33), (38, 34), (39, 34), (39, 35), (40, 35), (41, 35), (41, 36), (41, 37), (42, 37), (42, 38), (42, 39), (43, 39), (44, 39), (45, 39), (46, 39), (47, 39), (48, 39), (49, 39), (50, 39), (51, 39), (52, 39), (52, 40), (53,

In [13]:
def findNeighboringOpenCoordsIncludingDiagonals(coords, maze):
    cellRow, cellCol = coords
    neighbors = findNeighboringOpenCoords(coords, maze)
    potentialDiagonalNeighbors = [(cellRow - 1, cellCol - 1), (cellRow - 1, cellCol + 1), (cellRow + 1, cellCol - 1), (cellRow + 1, cellCol + 1)]
    for potentialDiagonalNeighbor in potentialDiagonalNeighbors:
        row, col = potentialDiagonalNeighbor
        if (row >= maze.dim or row < 0 or col >= maze.dim or col < 0 or maze.board[row,col] != Cell.OPEN):
            continue
        neighbors.append((row, col))
    return neighbors

def diagonalTravelShortestPathLength(coords, maze, visited):
    cellRow, cellCol = coords 
    shortestPathLength = 0
    expandedCells = 0
    if ((cellRow, cellCol)) not in visited:
        shortestPath, expandedCellsBySearch = shortestPathSearch(maze, startCoords = (cellRow, cellCol), findNeighborsFunction = findNeighboringOpenCoordsIncludingDiagonals)
        shortestPathLength = len(shortestPath)
        expandedCells = expandedCellsBySearch
        for i in range(0, shortestPathLength):
            row, col = shortestPath[i]
            if ((row, col)) in visited: 
                break
            else:
                visited[(row, col)] = shortestPathLength - i - 1
    else:
        shortestPathLength = visited[(cellRow, cellCol)]
    return shortestPathLength, expandedCells

if (solvable):
    startTime = time.time()
    shortestPath, expandedCells = shortestPathSearch(maze, heuristicFunction = diagonalTravelShortestPathLength)
    print("Expanded cells:", expandedCells)
    print("Shortest path length:", len(shortestPath), "\nShortest path:", shortestPath)
    boardWithPath = copy.deepcopy(maze.board)
    for coords in shortestPath:
        row, col = coords
        if (maze.board[row, col] == -1):
            print("Error")

Expanded cells: 388265
Shortest path length: 81 
Shortest path: [(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (3, 3), (3, 4), (4, 4), (5, 4), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (6, 12), (5, 12), (5, 13), (5, 14), (6, 14), (6, 15), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (8, 19), (8, 20), (8, 21), (9, 21), (9, 22), (10, 22), (11, 22), (12, 22), (13, 22), (13, 23), (14, 23), (15, 23), (16, 23), (17, 23), (17, 24), (18, 24), (18, 25), (18, 26), (18, 27), (19, 27), (20, 27), (20, 28), (21, 28), (22, 28), (23, 28), (23, 29), (24, 29), (24, 30), (25, 30), (25, 31), (25, 32), (26, 32), (27, 32), (28, 32), (29, 32), (30, 32), (31, 32), (31, 33), (32, 33), (33, 33), (33, 34), (34, 34), (35, 34), (35, 35), (35, 36), (36, 36), (36, 37), (36, 38), (37, 38), (37, 39), (38, 39), (39, 39)]


In [ ]:
differentPs = range(0.5, 0.55, 0.05)